In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import warnings

In [48]:
# Assumes the run outputs have been copied over
base = pathlib.Path("runs")
run_direcs = list(d for d in base.iterdir() if d.is_dir())
run_direcs.sort(
    # Sort by run number
    key=lambda fn: int(str(fn).split("-")[3][3:])
)

In [49]:
# Copy the energy info from the macro script
# Energy parameters: start, end, step
ea = 0.0
eb = 20
de = 0.1

# How many runs per energy range
num_runs = 100000
energy_bins = np.arange(ea, eb + de, de)

hits = list()
with warnings.catch_warnings():
    # Ignore empty file warnings
    warnings.simplefilter("ignore")
    for direc in run_direcs:
        try:
            _, energies = np.loadtxt(direc / "cryst-out.tab", unpack=True)
        except ValueError:
            # Sometimes, the file is empty
            energies = []
        hits.append(energies)

In [50]:
# Now we can bin the hits up into a response matrix
# We always need to start with a square matrix
response_matrix = np.zeros((energy_bins.size - 1, energy_bins.size - 1))
for i, h in enumerate(hits):
    response_matrix[:, i], _ = np.histogram(
        h, bins=energy_bins
    )

    # Divide by the total number of incident photons: each bin turns into a probability
    response_matrix[:, i] /= num_runs

In [51]:
import matplotlib.colors as mcol

# Plot the response
%matplotlib qt
plt.style.use('nice.mplstyle')
fig, ax = plt.subplots(figsize=(8, 6))

norm = mcol.LogNorm(vmin=1e-5, vmax=1)
cmap = plt.get_cmap("plasma").copy()
cmap.set_bad("black")

pcm = ax.pcolormesh(
    energy_bins,
    energy_bins,
    response_matrix,
    norm=norm,
    cmap=cmap
)

ax.set(
    title='100 um of Al and 500 um of Si',
    xlabel='Count energy (keV)',
    ylabel='Photon energy (keV)',
    aspect=1.0
)
fig.colorbar(pcm, label='Conversion probability')

plt.show()

In [52]:
# The effective area sums along one axis of the response matrix
geometric_area = (0.1 * 0.1) * np.pi
effective_area = geometric_area * response_matrix.sum(axis=0)

fig, ax = plt.subplots()
ax.stairs(effective_area, energy_bins)
ax.set(
    xscale='linear',
    yscale='log',
    title='Effective area, 100um Al 500um Si',
    xlabel='Energy (keV)',
    ylabel='Area (cm$^2$)',
    ylim=(1e-3, None)
)
plt.show()